In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [2]:
url_front = 'http://data.insideairbnb.com/singapore/sg/singapore/'
url_end_listings = '/data/listings.csv.gz'
url_end_reviews = '/data/reviews.csv.gz'

# Combine Data

In [3]:
dates = ['2019-03-18', '2019-04-20', '2019-05-23', '2019-06-25', '2019-07-21', '2019-08-28', '2019-09-25', '2019-10-25', '2019-11-26', '2019-12-28',
         '2020-01-26', '2020-02-27', '2020-03-21', '2020-04-26', '2020-05-27', '2020-06-22', '2020-10-26', '2020-11-27', '2020-12-29','2021-01-27']

In [4]:
# Keep common features across all months (from Oct, 2020 onwards some more features were added)
listings_2019_03 = pd.read_csv(url_front+'2019-03-18'+url_end_listings)
listings_2020_10 = pd.read_csv(url_front+'2020-10-26'+url_end_listings)

In [5]:
long_features = listings_2019_03.columns.values
short_features = listings_2020_10.columns.values
keep = [x for x in long_features if x in short_features]

In [6]:
keep

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availability_60',
 'avail

In [7]:
delete_col = ['listing_url',
 'scrape_id',
 'last_scraped',
 'picture_url',
 'host_url',
 'host_name',
 'host_location',
 'host_acceptance_rate',
 'host_thumbnail_url',
 'host_picture_url',
 'host_listings_count',
 'host_total_listings_count',
 'neighbourhood',
 'has_availability',
 'calendar_last_scraped',
 'first_review',
 'last_review',
 'license',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms']

In [8]:
def read_dataset(date):
    reviews = pd.read_csv(url_front+date+url_end_reviews).drop(columns = ['id'])
    listings = pd.read_csv(url_front+date+url_end_listings)[keep].drop(columns = delete_col) 

    return reviews, listings

In [9]:
def anti_join(x, y, date, on):
    # Find comments for new listings.
    merged = pd.merge(left=x, right=y, how='left', indicator=True, on=on)
    left_only = merged.loc[merged._merge == 'left_only', :].drop(columns='_merge')
    ans_1 = left_only.drop(
        columns = ['date_y', 'reviewer_id_y', 'reviewer_name_y','comments_y']).rename(
        columns = lambda x: x.replace('_x', ''))
    
    # Find new comments for existing listings.
    left_right = merged.loc[(merged._merge == 'both') & (merged.date_x >= date), :].drop(columns='_merge')
    ans_2 = left_right.drop(
        columns = ['date_y', 'reviewer_id_y', 'reviewer_name_y','comments_y']).rename(
        columns = lambda x: x.replace('_x', ''))
    
    ans = pd.concat([ans_1,ans_2]).drop_duplicates()
    return ans

In [10]:
reviews, listings = read_dataset(dates[0])
data = pd.merge(reviews, listings, left_on = 'listing_id', right_on = 'id', how = 'left').drop(columns = 'id')   

In [11]:
for i in range (1, 19):
    date = dates[i]
    this_review, this_listings = read_dataset(date)
    new_review = anti_join(this_review, reviews, dates[i-1], 'listing_id')
    new_review = new_review.drop_duplicates()
    new_data = pd.merge(new_review, this_listings, left_on = 'listing_id', right_on = 'id', how = 'left').drop(columns = 'id') 
    
    # concat the data 
    reviews = pd.concat([reviews, new_review]).drop_duplicates()
    data = pd.concat([data, new_data]).drop_duplicates()
    
    print('finish', i)

finish 1


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (42,43,61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


finish 2
finish 3
finish 4
finish 5
finish 6
finish 7
finish 8
finish 9
finish 10
finish 11
finish 12
finish 13
finish 14
finish 15
finish 16
finish 17
finish 18


In [12]:
data.comments.isna().value_counts()

False    155078
True        172
Name: comments, dtype: int64

In [13]:
reviewer_id = data.pop('reviewer_id')
data.insert(0, 'reviewer_id', reviewer_id)
data.head()

,reviewer_id,listing_id,date,reviewer_name,comments,name,description,neighborhood_overview,host_id,host_since,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month
0,8557223,49091,2013-10-21,Jared,Fran was absolutely gracious and welcoming. Ma...,COZICOMFORT LONG TERM STAY ROOM 2,This is Room No. 2.(available after Jan 15 201...,NaN,266763,2010-10-20,...,94.0,10.0,10.0,10.0,10.0,8.0,8.0,f,2,0.02
1,1356099,50646,2014-04-18,James,A comfortable room in a smart condo developmen...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,91.0,9.0,10.0,10.0,10.0,9.0,9.0,f,1,0.30
2,15222393,50646,2014-06-05,Welli,Stayed over at Sujatha's house for 3 good nigh...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,91.0,9.0,10.0,10.0,10.0,9.0,9.0,f,1,0.30
3,5543172,50646,2014-07-02,Cyril,It's been a lovely stay at Sujatha's. The room...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,91.0,9.0,10.0,10.0,10.0,9.0,9.0,f,1,0.30
4,817532,50646,2014-07-08,Jake,"We had a great experience. A nice place, an am...",Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,91.0,9.0,10.0,10.0,10.0,9.0,9.0,f,1,0.30


# Find Nearby Sites

In [42]:
# Extract the location data.
df_lat_lon = data[['latitude','longitude']].reset_index(drop=True)

In [79]:
# Read in the locations of popular sites
df_site = pd.read_csv('../Datasets/popular_sites.csv')
df_site.head()

,Title,Image,kfi5wf,jdzyld,nfofm
0,Marina Bay Sands Singapore,NaN,4.5,"(7,026)",High-end hotel with bay views & dining
1,Gardens by the Bay,NaN,4.7,"(107,063)",Modern public gardens on reclaimed land
2,Sentosa,NaN,4.6,"(3,415)",Palawan Beach & Tiger Sky Tower
3,Merlion,NaN,4.6,"(25,932)","Half-lion, half-fish statue & fountain"
4,Universal Studios Singapore,NaN,4.6,"(77,742)",Movie- & TV-themed amusement park


In [16]:
# Read in the locations of nearby MRT Stations
df_mrt = pd.read_csv('../Datasets/sg_mrt.csv')
df_mrt.head()

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388093,103.905418,OTHERS
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380018,103.772667,OTHERS


In [80]:
# Rename some attributes for easier recognization.
df_site.rename(columns={'Title':'site_name','nfofm':'intro','kfi5wf':'rating','jdzyld':'num_ratings'},inplace=True)
df_mrt.rename(columns={'STN_NAME':'station_name','STN_NO':'station_no','Latitude':'latitude','Longitude':'longitude','COLOR':'color'},inplace=True)

In [53]:
df_site.head()

,site_name,Image,rating,num_ratings,intro,longitude,latitude
0,Marina Bay Sands Singapore,NaN,4.5,"(7,026)",High-end hotel with bay views & dining,103.861005919029,1.28466203640592
1,Gardens by the Bay,NaN,4.7,"(107,063)",Modern public gardens on reclaimed land,103.868661506352,1.29093310289853
2,Sentosa,NaN,4.6,"(3,415)",Palawan Beach & Tiger Sky Tower,103.823350504845,1.25830214200274
3,Merlion,NaN,4.6,"(25,932)","Half-lion, half-fish statue & fountain",103.854558779513,1.28638676066506
4,Universal Studios Singapore,NaN,4.6,"(77,742)",Movie- & TV-themed amusement park,103.820263350055,1.2550924149692


In [82]:
df_site.num_ratings = df_site.num_ratings.dropna().apply(lambda x: int(x[2:-1].replace(',','')))

In [100]:
# Rate popularity, calculated using Bayesian Probability, 
# by choosing the expected value of rating R and expected number of ratings W.
R = df_site[df_site.num_ratings >= 1000].rating.median()
W = df_site.num_ratings.mean() / 5

df_site['norm_rating'] = (df_site.rating * df_site.num_ratings + R * W) / (df_site.num_ratings + W)
df_site.norm_rating.fillna(4.5) # The missing data is at Clark Quey whose rating was missing and then imputed manually.

0     4.500000
1     4.696903
2     4.566973
3     4.593902
4     4.597880
        ...   
94    4.500000
95    4.360272
96    4.427001
97    4.368015
98    4.465705
Name: norm_rating, Length: 99, dtype: float64

In [103]:
df_site['norm_rating'].describe()

count    98.000000
mean      4.486507
std       0.110920
min       4.133485
25%       4.437528
50%       4.500000
75%       4.565143
max       4.696903
Name: norm_rating, dtype: float64

In [110]:
# Find the geolocation of these sites using external tools.
def find_geolocation(df,name):
    import json
    import requests

    longs = []
    lats = []
    for title in df[name]:
      url= 'https://developers.onemap.sg/commonapi/search?searchVal='+ '%20'.join(title.split()) +'&returnGeom=Y&getAddrDetails=Y&pageNum=1'
      response = requests.get(url)
      obj = response.json()
      if not obj['results']:
        longs.append('')
        lats.append('')
        continue
      longs.append(obj['results'][0]['LONGITUDE'])
      lats.append(obj['results'][0]['LATITUDE'])
    return longs, lats

In [111]:
longs,lats = find_geolocation(df_site,'site_name')
df_site['longitude'] = np.array(longs)
df_site['latitude'] = np.array(lats)

In [112]:
df_site = df_site[df_site.longitude != ''] # ignore unsuccessful queries.

In [113]:
# Function to find the distance between two coordinates.
from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...

In [224]:
# Reusable functions for finding n(default=3) nearest sites of each listed hotel.
def find_nearby(df, df_nearby, name, thres=3):
  distances = []
  for i in range(df.shape[0]):
    lat1 = float(df.latitude[i])
    lon1 = float(df.longitude[i])
    candidates = []
    for j in range(df_nearby.shape[0]):
        lat2 = float(df_nearby.latitude.iloc[j])
        lon2 = float(df_nearby.longitude.iloc[j])
        dist = distance(lat1, lon1, lat2, lon2)
        candidates.append([df_nearby[name].iloc[j],dist])
    candidates.sort(key=lambda x:x[1])
    distances.append(candidates[:thres])
  return distances

In [237]:
def avg_rate(x,thres,df,feature_n,rating_n):
    from math import sqrt
    
    sum_dist = 0
    sum_rating = 0
    for i in range(thres):
        sum_dist += x[i][1]
        sum_rating += df.loc[df[feature_n] == x[i][0], rating_n].values[0]
    return sqrt(sum_rating/sum_dist)

In [204]:
distances_site = find_nearby(df_lat_lon, df_site, 'site_name')
data['nearby_sites_rating'] = np.array([avg_rate(x,3,df_site,'site_name','norm_rating') 
                                 for x in distances_site])

In [239]:
distances_mrt = find_nearby(df_lat_lon, df_mrt,'station_name',1)
data['nearest_station_dist'] = np.array([x[1] for x in distances_mrt])

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [243]:
data['nearest_station_dist'] = np.array([x[0][1] for x in distances_mrt])

In [244]:
data.head()

,reviewer_id,listing_id,date,reviewer_name,comments,name,description,neighborhood_overview,host_id,host_since,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,nearby_sites_1km,nearby_sites,nearby_sites_rating,nearest_station_dist
0,8557223,49091,2013-10-21,Jared,Fran was absolutely gracious and welcoming. Ma...,COZICOMFORT LONG TERM STAY ROOM 2,This is Room No. 2.(available after Jan 15 201...,NaN,266763,2010-10-20,...,10.0,8.0,8.0,f,2,0.02,[],0.991480,0.995731,0.617762
1,1356099,50646,2014-04-18,James,A comfortable room in a smart condo developmen...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,[],1.611899,1.269606,0.399696
2,15222393,50646,2014-06-05,Welli,Stayed over at Sujatha's house for 3 good nigh...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,[],1.611899,1.269606,0.399696
3,5543172,50646,2014-07-02,Cyril,It's been a lovely stay at Sujatha's. The room...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,[],1.611899,1.269606,0.399696
4,817532,50646,2014-07-08,Jake,"We had a great experience. A nice place, an am...",Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,[],1.611899,1.269606,0.399696


In [245]:
data.to_csv('../Datasets/cleaned_customer.csv',index=False)